In [2]:
import pprint
import requests
import sys
import pandas as pd
from datetime import datetime

api_key = "9b656c6f5237727cdf1e35288a7afa8c"
geo_URL = 'http://api.openweathermap.org/geo/1.0/direct'

loc = [
  'Buenos Aires, Argentina',
  'Guangzhou, China',
  'Wichita, Kansas',
  'Niskayuna, New York',
  'Gwangmyeong, South Korea',
  'Taipei, Taiwan',
  'Nanaimo, British Columbia',
  'Chennai, India',
  'Barrington, Illinois',
  'Littleton, Colorado',
  'Peterhead, Scotland',
  'Vizag, India',
  'Des Moines, Iowa',
  'Beijing, China',
  'Killeen, Texas',
  'Morehead City, North Carolina'
]

df_wide = pd.DataFrame(index=loc)
######

for city in loc:
    geo = f'{geo_URL}?q={city}&limit=5&appid={api_key}'
    resp = requests.get( geo )

    if resp.status_code != 200: # Failure?
        print( f'Error geocoding {city}: {resp.status_code}' )
        sys.exit( 1 )

    # OpenWeatherMap returns a list of matching cities, up to the limit specified
    # in the API call; even if you only ask for one city (limit=5), it's still
    # returned as a 1-element list

    if len( resp.json() ) == 0: # No such city?
        print( f'Error locating city {city}; {resp.status_code}' )
        sys.exit( 2 )

    json = resp.json()
    if type( json ) == list: # List of cities?
        lat = json[ 0 ][ 'lat' ]
        lon = json[ 0 ][ 'lon' ]
    else: # Unknown city?
        print( f'Error, invalid data returned for city {city}, {resp.status_code}' )
        sys.exit( 3 )

    # Use Peterhead's latitude and longitude to get its 5-day forecast in 3-hour blocks

    forecast_URL = 'http://api.openweathermap.org/data/2.5/forecast'
    forecast = f'{forecast_URL}?lat={lat}&lon={lon}&appid={api_key}'
    resp = requests.get( forecast )

    if resp.status_code != 200: # Failure?
        print( f'Error retrieving data: {resp.status_code}' )
        sys.exit( 4 )

    data = resp.json()
##### 

    data_list = []
    for i in range(len(data['list'])):
        forecast_data = data[ 'list' ][ i ]
        dt = datetime.strptime(forecast_data['dt_txt'], '%Y-%m-%d %H:%M:%S')
        temp_max = forecast_data['main']['temp_max']
        temp_min = forecast_data['main']['temp_min']
        data_list.append({
            'city': city,
            'datetime': dt,
            'temp_max': temp_max,
            'temp_min': temp_min
        })

    df = pd.DataFrame(data_list)
#####

    df_grouped = df.groupby(pd.Grouper(key='datetime', axis=0, freq='D')).aggregate({
        'city': lambda x: x.iloc[0],
        'temp_max': 'max',
        'temp_min': 'min'})
   
    df_grouped['min_avg'] = df_grouped['temp_min'].mean()
    df_grouped['max_avg'] = df_grouped['temp_max'].mean()
#####

    for i, day in enumerate(df_grouped.index, start=0):
        if i>4:
            break
        df_wide.loc[city, f'Min_{i}'] = df_grouped.loc[day, 'temp_min']
        df_wide.loc[city, f'Max_{i}'] = df_grouped.loc[day, 'temp_max']


df_wide = df_wide.drop(columns=['Min_0', 'Max_0'])
df_wide = df_wide.iloc[:] - 273.15
df_wide = df_wide.reset_index()
df_wide = df_wide.rename({'index': 'City'}, axis='columns')
df_wide['Min_Avg'] = df_wide[['Min_1', 'Min_2', 'Min_3', 'Min_4']].mean(axis=1).round(2)
df_wide['Max_Avg'] = df_wide[['Max_1', 'Max_2', 'Max_3', 'Max_4']].mean(axis=1).round(2)
df_wide

,City,Min_1,Max_1,Min_2,Max_2,Min_3,Max_3,Min_4,Max_4,Min_Avg,Max_Avg
0,"Buenos Aires, Argentina",11.65,14.60,9.03,9.99,10.39,13.42,11.59,16.16,10.67,13.54
1,"Guangzhou, China",29.04,35.89,28.73,34.60,28.71,37.07,26.95,32.22,28.36,34.95
2,"Wichita, Kansas",19.89,32.33,19.86,31.13,19.85,31.09,18.34,28.98,19.49,30.88
3,"Niskayuna, New York",12.94,26.79,13.88,27.91,15.30,28.34,16.04,30.64,14.54,28.42
4,"Gwangmyeong, South Korea",24.56,28.82,22.74,24.49,24.88,29.00,25.30,29.53,24.37,27.96
5,"Taipei, Taiwan",27.57,36.48,28.78,37.25,27.01,34.82,27.09,35.90,27.61,36.11
6,"Nanaimo, British Columbia",15.95,22.49,16.95,23.16,17.65,26.93,15.71,26.25,16.57,24.71
7,"Chennai, India",27.53,29.16,28.22,34.08,27.88,35.30,28.44,33.79,28.02,33.08
8,"Barrington, Illinois",13.08,24.13,14.98,24.86,17.59,26.42,17.64,26.90,15.82,25.58
9,"Littleton, Colorado",18.80,31.27,18.65,30.30,16.78,24.23,16.07,28.17,17.58,28.49


In [3]:
df_wide[:][2:5]

,City,Min_1,Max_1,Min_2,Max_2,Min_3,Max_3,Min_4,Max_4,Min_Avg,Max_Avg
2,"Wichita, Kansas",19.89,32.33,19.86,31.13,19.85,31.09,18.34,28.98,19.49,30.88
3,"Niskayuna, New York",12.94,26.79,13.88,27.91,15.30,28.34,16.04,30.64,14.54,28.42
4,"Gwangmyeong, South Korea",24.56,28.82,22.74,24.49,24.88,29.00,25.30,29.53,24.37,27.96
